In [1]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install -q -U sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-d74nt1z2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-d74nt1z2
  Resolved https://github.com/huggingface/transformers to commit 9ed538f2e67ee10323d96c97284cf83d44f0c507
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.

In [2]:
!git clone https://github.com/leanprover-community/mathlib4.git || echo "already cloned"

fatal: destination path 'mathlib4' already exists and is not an empty directory.
already cloned


In [3]:
from transformers import LlamaForCausalLM, CodeLlamaTokenizer, BitsAndBytesConfig
import json
import pandas as pd
import os
from transformers import AutoTokenizer

base_model_name = "codellama/CodeLlama-7b-hf"
# You can initialize the tokenizer here. For example, for BERT:
tokenizer = CodeLlamaTokenizer.from_pretrained(base_model_name)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


# Download the dataset

In [1]:
!mkdir data_simple || echo "data_simple dir already exists"
DATA_URL = "https://github.com/Lev-Stambler/math_lean_for_code_llama/raw/main/data_simple/test.json"
!(cd data_simple && wget $DATA_URL)

mkdir: cannot create directory ‘data_simple’: File exists
data_simple dir already exists
--2023-10-02 15:47:19--  https://github.com/Lev-Stambler/math_lean_for_code_llama/raw/main/data_simple/test.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/Lev-Stambler/math_lean_for_code_llama/main/data_simple/test.json [following]
--2023-10-02 15:47:19--  https://media.githubusercontent.com/media/Lev-Stambler/math_lean_for_code_llama/main/data_simple/test.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248697412 (237M) [application/octet-stream]
Saving to: ‘test.json’

test.json  

In [6]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={'train': ["data_simple/test.json"]})['train']
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 53315
})

In [8]:


# When prompted, paste the HF access token you created earlier.
# from huggingface_hub import notebook_login
# notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# dataset_name = "<your_hf_dataset>"
# dataset = load_dataset(dataset_name, split="train")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    # Added by Lev, remove
    bnb_4bit_use_double_quant=True,
)

device_map = {"": 0}

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,

    # Added by Lev, remove
    load_in_4bit=True,
    # use_auth_token=True
)
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf",
#     device_map='auto',
#     # load_in_4bit=True,
#     device_map=device_map,
#     trust_remote_code=True,
#     quantization_config=bnb_config
# )
base_model = model
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

# max_seq_length = 1_024 # TODO: TUNE!
max_seq_length = 512 # TODO: TUNE!

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)
# TODO: now we are just out of memory...

2023-10-02 14:06:37.804065: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 14:06:37.915744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 14:06:38.356607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/lev/.local/lib/python3.9/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/53315 [00:00<?, ? examples/s]

/home/lev/.local/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARAL

You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 7.79 GiB total capacity; 6.28 GiB already allocated; 54.94 MiB free; 6.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF